# Podstawy Przetwarzania Dużych Zbiorów Danych

Grupa IBD4 Poniedziałki 8:30

Łukasz Moskwa 247739 

Artur Szewczykowski 252546



# Część 1
 
1. https://github.com/GoogleCloudPlatform/covid-19-open-data/blob/main/docs/table-by-sex.md
2. https://medium.com/artificial-intelligence-101/data-analysis-with-google-bigquery-exploring-the-covid-19-open-data-public-dataset-2d246c66422d

# Część 2

In [38]:
# 2.5 Uruchom klienta BigQuery z poziomu Pythona.

import os
from operator import index

import pandas as pd
from google.cloud import bigquery
 

In [39]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=".env/propane-flag-452609-u3-1feb9eb63dc4.json" # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client() 


In [3]:
# 2.6 Pobierz dane do obiektu DataFrame.

query = ('select * from bigquery-public-data.covid19_open_data.covid19_open_data limit 10')
query_job = client.query(query)    
query_result = query_job.result()  
df = query_result.to_dataframe()
df

,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,...,cumulative_vaccine_doses_administered_pfizer,new_persons_fully_vaccinated_moderna,cumulative_persons_fully_vaccinated_moderna,new_vaccine_doses_administered_moderna,cumulative_vaccine_doses_administered_moderna,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,location_geometry
0,AD,2020-01-16,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
1,AD,2021-12-28,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
2,AD,2021-06-13,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
3,AD,2020-09-03,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
4,AD,2021-05-28,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
5,AD,2020-10-19,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
6,AD,2020-04-20,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
7,AD,2021-03-31,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
8,AD,2021-05-26,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)
9,AD,2020-08-22,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,AD,AND,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(1.555278 42.558333)


# Część 3
Misja dodatkowa - Poznanie danych

In [4]:
# 3.1  Sprawdź, ile jest zapisanych wierszy z danymi. 

dataset_ref = client.dataset('covid19_open_data', project='bigquery-public-data')
table_ref = dataset_ref.table('covid19_open_data')
table = client.get_table(table_ref)

rows = table.num_rows

rows

22756333

In [51]:
# 3.2 Sprawdź, ile krajów jest uwzględnionych w danych. 

query = ("""
SELECT COUNT(DISTINCT country_code) AS num_countries
FROM bigquery-public-data.covid19_open_data.covid19_open_data

""")

query_job = client.query(query)
result = query_job.result()

number_of_country = result.to_dataframe()

# Zwraca 246, ponieważ niektóre tereny podległe takie jak "French Southern Territories", "Guam" itd.
# Co jest niby okey ale nie są to państwa bo ich jest 195
number_of_country.iloc[0, 0]


246

In [58]:
 # 3.3. Sprawdź, w jaki sposób zapisywane są dzienne informacje dla krajów.
 
# Dodają najczęsciej raz dziennie ale zdarza się, że kraj ma 2 wpisy ponieważ robi to dla innego aggregation_level
query = ("""
SELECT *
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE  aggregation_level = 0 AND country_name = 'Poland'
ORDER BY date ASC 
""")

country_all = client.query(query).to_dataframe()
country_all['date']

0      2020-01-01
1      2020-01-02
2      2020-01-03
3      2020-01-04
4      2020-01-05
          ...    
986    2022-09-13
987    2022-09-14
988    2022-09-15
989    2022-09-16
990    2022-09-17
Name: date, Length: 991, dtype: dbdate

In [59]:

is_continuous = (country_all['date'].diff().dropna() == pd.Timedelta(days=1)).all()

is_continuous


True

### 3.4. Sprawdź, w jaki sposób zapisywane są wartości liczbowe.

In [60]:
display(country_all.dtypes.value_counts())

float64    441
Int64      217
object      42
dbdate       1
Name: count, dtype: int64

Przykłady kolumn, które powiny być typem liczbowym a są obiektami czyli ciągami znaków.

In [61]:
# Wszystkie age_bins{1-9}
display(country_all[['population_largest_city', 'population_clustered', 'human_capital_index', 'area_rural_sq_km', 'area_urban_sq_km', 'life_expectancy']])

,population_largest_city,population_clustered,human_capital_index,area_rural_sq_km,area_urban_sq_km,life_expectancy
0,1775933,1775933,0.747,277840,30501,77.7
1,1775933,1775933,0.747,277840,30501,77.7
2,1775933,1775933,0.747,277840,30501,77.7
3,1775933,1775933,0.747,277840,30501,77.7
4,1775933,1775933,0.747,277840,30501,77.7
...,...,...,...,...,...,...
986,1775933,1775933,0.747,277840,30501,77.7
987,1775933,1775933,0.747,277840,30501,77.7
988,1775933,1775933,0.747,277840,30501,77.7
989,1775933,1775933,0.747,277840,30501,77.7


In [62]:
# Zapisane w typie coś / 1000
display(country_all.filter(like='_per_'))

,gdp_per_capita_usd,nurses_per_1000,physicians_per_1000,hospital_beds_per_1000
0,15222,6.8926,2.3788,NaN
1,15222,6.8926,2.3788,NaN
2,15222,6.8926,2.3788,NaN
3,15222,6.8926,2.3788,NaN
4,15222,6.8926,2.3788,NaN
...,...,...,...,...
986,15222,6.8926,2.3788,NaN
987,15222,6.8926,2.3788,NaN
988,15222,6.8926,2.3788,NaN
989,15222,6.8926,2.3788,NaN


###  3.5. Sprawdź, jaki przedział czasowy jest uwzględniony w danych. Dodatkowo porównaj przedziały czasowe dla przypadków nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób w danych.

In [63]:


query_new_confirmed = ("""
SELECT date, SUM(new_confirmed)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
GROUP BY date
HAVING SUM(new_confirmed) IS NOT NULL AND SUM(new_confirmed) > 0
ORDER BY date ASC
""")

query_new_deceased = ("""
SELECT date, SUM(new_deceased)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
GROUP BY date
HAVING SUM(new_deceased) IS NOT NULL AND SUM(new_deceased) > 0
ORDER BY date ASC
""")

query_new_person_vaccinated = ("""
SELECT date, SUM(new_persons_vaccinated)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
GROUP BY date
HAVING SUM(new_persons_vaccinated) IS NOT NULL AND SUM(new_persons_vaccinated) > 0
ORDER BY date ASC
""")

new_confirmed = client.query(query_new_confirmed).to_dataframe()
new_deceased = client.query(query_new_deceased).to_dataframe()
new_person_vaccinated = client.query(query_new_person_vaccinated).to_dataframe()

In [64]:
print(f"Przedział czasowy danych jest od {country_all['date'].min()} do {country_all['date'].max()}.")
print(f"Przedział czasowy new_confirmed jest od {new_confirmed['date'].min()} do {new_confirmed['date'].max()}.")
print(f"Przedział czasowy new_deceased jest od {new_deceased['date'].min()} do {new_deceased['date'].max()}.")
print(f"Przedział czasowy new_person_vaccinated jest od {new_person_vaccinated['date'].min()} do {new_person_vaccinated['date'].max()}.")


Przedział czasowy danych jest od 2020-01-01 do 2022-09-17.
Przedział czasowy new_confirmed jest od 2020-01-01 do 2022-09-15.
Przedział czasowy new_deceased jest od 2020-01-02 do 2022-09-15.
Przedział czasowy new_person_vaccinated jest od 2020-12-07 do 2022-09-14.


###  3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń. 


1. Ile kolumn liczy zbiór danych

In [65]:
num_columns = country_all.shape[1]
print(f"Liczba kolumn: {num_columns}")

Liczba kolumn: 701


2. Które państwa podawały dane z podziałem na płeć

In [66]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_confirmed_male) AS confirmed_male, SUM(new_confirmed_female) AS confirmed_female, SUM(new_deceased_male) AS deceased_male, SUM(new_deceased_female) as deceased_female, SUM(new_tested_male) AS tested_male, SUM(new_tested_female) AS tested_female, SUM(new_hospitalized_patients_male) AS hospitalized_male, SUM(new_hospitalized_patients_female) AS hospitalized_female, SUM(new_intensive_care_patients_male) AS cared_male, SUM(new_intensive_care_patients_female) AS cared_female, SUM(new_recovered_male) AS recovered_male, SUM(new_recovered_female) AS recovered_female
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_confirmed_male) IS NOT NULL OR SUM(new_deceased_male) IS NOT NULL OR SUM(new_tested_male) IS NOT NULL OR SUM(new_hospitalized_patients_male) IS NOT NULL OR SUM(new_recovered_male) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

,Country_name,3_letter_code,confirmed_male,confirmed_female,deceased_male,deceased_female,tested_male,tested_female,hospitalized_male,hospitalized_female,cared_male,cared_female,recovered_male,recovered_female
0,Argentina,ARG,4418859,4780139,72801,53930,13178569,14928395,248464,222232,48422,33209,<NA>,<NA>
1,Brazil,BRA,8875904,8368673,311308,244105,26382353,26921275,996777,792931,330847,245941,0,0
2,Colombia,COL,2697358,3042821,93611,62159,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2542476,2898545
3,Czech Republic,CZE,1956993,2111418,23149,17783,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1925067,2082491
4,Estonia,EST,272596,326685,<NA>,<NA>,1595674,1890648,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,Finland,FIN,526714,587525,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,Germany,DEU,6092181,6356325,64006,56007,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4370388,4550358
7,Hong Kong,HKG,7417,<NA>,127,<NA>,<NA>,<NA>,6708,<NA>,<NA>,<NA>,<NA>,<NA>
8,India,IND,92449,51846,30,13,<NA>,<NA>,3431,1712,<NA>,<NA>,<NA>,<NA>
9,Malaysia,MYS,1676866,1323134,20855,15444,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


3. Które państwa podawały dane z podziałem na kategorie wiekowe

In [67]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_confirmed_age_0) AS confirmed_age_0, SUM(new_deceased_age_0) AS deceased_age_0, SUM(new_tested_age_0) AS tested_age_0, SUM(new_hospitalized_patients_age_0) as hospitalized_age_0, SUM(new_intensive_care_patients_age_0) AS cared_age_0, SUM(new_recovered_age_0) AS recovered_age_0 
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_confirmed_age_0) IS NOT NULL OR SUM(new_deceased_age_0) IS NOT NULL OR SUM(new_tested_age_0) IS NOT NULL OR SUM(new_hospitalized_patients_age_0) IS NOT NULL OR SUM(new_recovered_age_0) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

,Country_name,3_letter_code,confirmed_age_0,deceased_age_0,tested_age_0,hospitalized_age_0,cared_age_0,recovered_age_0
0,Argentina,ARG,224240,192,1303477,41095,3269,<NA>
1,Brazil,BRA,891485,1214,2726462,21195,5273,0
2,Colombia,COL,198102,360,<NA>,<NA>,<NA>,191387
3,Czech Republic,CZE,321365,7,<NA>,<NA>,<NA>,320168
4,Estonia,EST,44622,<NA>,208798,<NA>,<NA>,<NA>
5,Finland,FIN,85684,<NA>,<NA>,<NA>,<NA>,<NA>
6,France,FRA,2789829,<NA>,23039639,<NA>,<NA>,<NA>
7,Germany,DEU,437029,23,<NA>,<NA>,<NA>,290555
8,Hong Kong,HKG,666,0,<NA>,578,<NA>,<NA>
9,India,IND,4273,1,<NA>,78,<NA>,<NA>


4. Które państwa podawały dane dotyczące szczepień z podziałem na rodzaj(firmę) szczepionki

In [68]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_persons_fully_vaccinated_moderna) AS vaccinated_moderna, SUM(new_persons_fully_vaccinated_janssen) AS vaccinated_janssen, SUM(new_persons_fully_vaccinated_pfizer) AS vaccinated_pfizer, SUM(new_persons_vaccinated) AS total_vaccinated 
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_persons_fully_vaccinated_moderna) IS NOT NULL OR SUM(new_persons_fully_vaccinated_janssen) IS NOT NULL OR SUM(new_persons_fully_vaccinated_pfizer) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

,Country_name,3_letter_code,vaccinated_moderna,vaccinated_janssen,vaccinated_pfizer,total_vaccinated
0,Malaysia,MYS,<NA>,<NA>,15153452,27880068
1,United States of America,USA,63469471,16886719,112344792,263883327


5. Jakie jest znaczenie wartośći liczbowych z kolumn dotyczących restrykcji (school_closing, workplace_closing, cancel_public_events, restrictions on gatherings)

In [69]:
query = ("""
SELECT DISTINCT date, country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, school_closing, workplace_closing, cancel_public_events, restrictions_on_gatherings
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
ORDER BY country_name
LIMIT 6
""")

df_temp= client.query(query).to_dataframe()
df_temp

,date,Country_name,3_letter_code,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings
0,2022-01-13,Afghanistan,AFG,0,2,1,0
1,2021-03-29,Afghanistan,AFG,0,2,2,0
2,2021-01-05,Afghanistan,AFG,1,0,0,0
3,2021-02-21,Afghanistan,AFG,1,0,0,0
4,2022-04-27,Afghanistan,AFG,0,2,1,0
5,2022-08-02,Afghanistan,AFG,<NA>,<NA>,<NA>,<NA>


<b>School closing:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane zamknięcie lub wprowadzenie obostrzeń,
<br> 2 - nakaz zamknięcia niektórych typów szkół,
<br> 3 - nakaz zamknięcia wszystkich rodzai szkół. <br> <br>
<b>Workplace closing:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane zamknięcie lub praca zdalna, albo wprowadzenie obostrzeń dla firm,
<br> 2 - nakaz zamknięcia niektórych sektorów lub obowiązkowa praca zdalna dla części pracowników,
<br> 3 - nakaz zamknięcia wszystkich miejsc pracy z wyjątkiem działalności niezbędnych (np. sklepy spożywcze. placówki medyczne). <br> <br>
<b>Cancel public events:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane unikanie/odwoływanie,
<br> 2 - zakaz organizacji. <br> <br>
<b>Restrictions on gatherings:</b>
<br> 0 - brak ograniczeń,
<br> 1 - ograniczenia dla bardzo dużych zgromadzeń (powyżej 1000 osób),
<br> 2 - ograniczenia dla zgromadzeń od 101 do 1000 osób,
<br> 3 - ograniczenia dla zgromadzeń od 11 do 100 osób,
<br> 4 - ograniczenia dla zgromadzeń do 10 osób lub mniej.
<br>
via https://github.com/OxCGRT/covid-policy-dataset/blob/main/documentation_and_codebook.md <br>

# Część 4
Przeanalizuj poniższe przypadki. Zastanów się, jakie dane potrzebujesz do każdego z nich, a następnie zapisz je w osobnych, jak najprostszych obiektach DataFrame. Na tym etapie nie przetwarzaj ich. Zadbaj o czystość danych, m.in. usuń niepotrzebne puste wartości i duplikaty, zunifikuj sposób podawania informacji, zaproponuj sposób naprawy błędnych danych. Gotowe dane z obiektów DataFrame zapisz w osobnych plikach CSV.


In [70]:
# 4.1 Chcemy posiadać podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.

query = ("""
SELECT DISTINCT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, population, area_sq_km, population_density, population_urban, gdp_per_capita_usd
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0
GROUP BY country_name, iso_3166_1_alpha_3, population, area_sq_km, population_density, population_urban, gdp_per_capita_usd
""")

country_data = client.query(query).to_dataframe()
country_data.to_csv("wyniki\\country_data.csv", index=False)

In [71]:
country_data

,Country_name,3_letter_code,population,area_sq_km,population_density,population_urban,gdp_per_capita_usd
0,Colombia,COL,50882884,1141748,45.861,40827302,6432
1,Germany,DEU,83019213,357580,234.700,64324835,47554
2,Spain,ESP,46937060,505935,93.100,37927409,30326
3,Hong Kong,HKG,7496988,1110,7139.989,7507400,48755
4,Malaysia,MYS,32655400,330345,98.512,24475766,11414
...,...,...,...,...,...,...,...
241,East Timor,TLS,1318442,14870,88.665,400182,1294
242,Tanzania,TZA,59734213,947300,67.435,20011885,1122
243,Ukraine,UKR,43733759,603550,75.492,30835699,3659
244,Saint Vincent and the Grenadines,VCT,110947,390,284.479,58185,7463


In [72]:
# 4.2 Chcemy wygenerować statystyki dotyczące zachorowań na COVID-19 na całym świecie.

# Tutaj są wartości null w sumie. Można podmienić na 0 COALESCE(SUM(new_confirmed), 0) ale oznaczała by to, że nikt tam nie chorował a naprawdę nie ma danych.
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`, new_confirmed as `Total_confirmed_cases`,
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_confirmed
ORDER BY date
""")

covid_statistic = client.query(query).to_dataframe()
covid_statistic.dropna(subset=['Total_confirmed_cases'], inplace=True)
covid_statistic.to_csv("wyniki\\covid_statistic.csv", index=False)


In [73]:
covid_statistic

,date,3_letter_code,Total_confirmed_cases
0,2020-01-01,IRQ,0
1,2020-01-01,MNG,0
2,2020-01-01,ECU,0
3,2020-01-01,SYC,0
4,2020-01-01,PAN,0
...,...,...,...
243240,2022-09-15,IDN,2651
243245,2022-09-15,ITA,17978
243246,2022-09-15,CHN,46558
243277,2022-09-15,MYS,2375


In [74]:
# 4.3 Chcemy poznać efekty COVID-19 poprzez uwypuklenie problemu śmiertelności ludzi spowodowanej wirusem.

query = ("""
SELECT date,iso_3166_1_alpha_3 AS `3_letter_code`, new_deceased as `Total_new_deceased` 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_deceased
""")

covid_deceased = client.query(query).to_dataframe()
covid_deceased.dropna(subset=['Total_new_deceased'], inplace=True)
covid_deceased.to_csv("wyniki\\covid_deceased.csv", index=False)


In [75]:
covid_deceased

,date,3_letter_code,Total_new_deceased
0,2020-10-29,COL,235
1,2021-08-09,COL,202
2,2022-01-09,FIN,25
3,2022-01-01,FRA,110
4,2020-08-28,FRA,19
...,...,...,...
243781,2021-05-15,AUT,15
243782,2021-06-12,BHR,15
243783,2022-07-06,NZL,15
243784,2021-06-19,GEO,15


In [76]:
# 4.4 Chcemy zaobserwować trendy i zależności dotyczące szczepień na COVID-19. 

query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`,
new_persons_vaccinated as `Total_new_vacinations`,
new_persons_fully_vaccinated as`New_persons_fully_vaccinated`
/*
SUM(new_persons_fully_vaccinated_pfizer) as `Total_fully_vacinations_by_pfizer`,
SUM(new_persons_fully_vaccinated_moderna) as `Total_fully_vacinations_by_moderna`,
SUM(new_persons_fully_vaccinated_janssen) as `Total_fully_vacinations_by_janssen`
*/
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_persons_vaccinated, new_persons_fully_vaccinated
""")

covid_trend_analysis = client.query(query).to_dataframe()
covid_trend_analysis.dropna(subset=['Total_new_vacinations', 'New_persons_fully_vaccinated'], how='all', inplace=True)
covid_trend_analysis.to_csv("wyniki\\covid_trend_analysis.csv", index=False)

In [77]:
covid_trend_analysis

,date,3_letter_code,Total_new_vacinations,New_persons_fully_vaccinated
0,2022-06-08,CZE,136,245
1,2021-10-31,CZE,3771,826
2,2022-05-29,EST,12,12
3,2021-07-14,PER,119686,30362
4,2021-11-05,PER,231357,60230
...,...,...,...,...
243781,2021-03-17,UKR,9832,0
243782,2020-10-29,BRA,<NA>,0
243783,2020-09-21,AUS,<NA>,0
243784,2020-04-09,AUS,<NA>,0


In [78]:
# 4.5 Zdefiniuj własny dodatkowy przypadek.

# Chcemy posidać informacje o investycji w rozwój szepionki oraz liczbe wykonanych testów
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`, CAST(new_tested AS INT64) as `Total_tested`
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, investment_in_vaccines, new_tested
ORDER BY date
""")

investment_in_vaccines = client.query(query).to_dataframe()
investment_in_vaccines.dropna(subset=['Total_tested'], how='all', inplace=True)
investment_in_vaccines.to_csv("wyniki\\investment_in_vaccines.csv", index=False)

display(investment_in_vaccines)

("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_tested_male) as `New_tested_male`, SUM(new_tested_female) as `New_tested_female` 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
GROUP BY country_name, iso_3166_1_alpha_3
ORDER BY country_name
""")


# Łóżka i pielęgniarki
("""
SELECT
    country_name AS Country_name, iso_3166_1_alpha_3 AS 3_letter_code, ((population * hospital_beds_per_1000) / 1000) AS Total_Hospital_Beds, 
((population * nurses_per_1000) / 1000) AS Total_Nurses
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE hospital_beds_per_1000 IS NOT NULL 
      AND nurses_per_1000 IS NOT NULL
GROUP BY country_name, iso_3166_1_alpha_3, population, hospital_beds_per_1000, nurses_per_1000
ORDER BY country_name;
""")



,date,3_letter_code,Total_tested
6,2020-01-01,ARG,4
30,2020-01-01,MEX,72
169,2020-01-01,BRA,1154
401,2020-01-02,ARG,42
403,2020-01-02,BRA,2337
...,...,...,...
243089,2022-09-15,AUT,67392
243147,2022-09-15,AUS,4367
243228,2022-09-15,MYS,0
243233,2022-09-15,ITA,146983


'\nSELECT\n    country_name AS Country_name, iso_3166_1_alpha_3 AS 3_letter_code, ((population * hospital_beds_per_1000) / 1000) AS Total_Hospital_Beds, \n((population * nurses_per_1000) / 1000) AS Total_Nurses\nFROM `bigquery-public-data.covid19_open_data.covid19_open_data`\nWHERE hospital_beds_per_1000 IS NOT NULL \n      AND nurses_per_1000 IS NOT NULL\nGROUP BY country_name, iso_3166_1_alpha_3, population, hospital_beds_per_1000, nurses_per_1000\nORDER BY country_name;\n'

# Część 5

Połącz ze sobą wszystkie dane otrzymane w części 4. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [83]:
merge = pd.merge(country_data, covid_statistic, on=['3_letter_code'])
merge = pd.merge(merge, covid_statistic, on=['date', '3_letter_code'])
merge = pd.merge(merge, covid_trend_analysis, on=['date', '3_letter_code'])
merge = pd.merge(merge, investment_in_vaccines, on=['date', '3_letter_code'])
merge.to_csv("wyniki\\merge.csv", index=False)
merge

,Country_name,3_letter_code,population,area_sq_km,population_density,population_urban,gdp_per_capita_usd,date,Total_confirmed_cases_x,Total_confirmed_cases_y,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested
0,Colombia,COL,50882884,1141748,45.861,40827302,6432,2021-03-14,2728,2728,54597,6306,30968
1,Colombia,COL,50882884,1141748,45.861,40827302,6432,2021-03-15,4440,4440,65338,5419,41720
2,Colombia,COL,50882884,1141748,45.861,40827302,6432,2021-03-16,5093,5093,59933,2243,34063
3,Colombia,COL,50882884,1141748,45.861,40827302,6432,2021-03-17,5802,5802,42313,5908,46016
4,Colombia,COL,50882884,1141748,45.861,40827302,6432,2021-03-18,6591,6591,51364,774,43986
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31513,South Africa,ZAF,59308690,1219090,48.891,39149717,6001,2022-06-09,1591,1591,16404,16781,20865
31514,South Africa,ZAF,59308690,1219090,48.891,39149717,6001,2022-06-13,710,710,838,854,10493
31515,South Africa,ZAF,59308690,1219090,48.891,39149717,6001,2022-06-14,1903,1903,14113,13788,22417
31516,South Africa,ZAF,59308690,1219090,48.891,39149717,6001,2022-06-20,1087,1087,566,654,14018


# Część 6


Usunięcie powtarzających się kolumn, usówamy z naszego zbioru, ponieważ zbiory znalezione potem okazują się być dokładniejsze.

In [84]:
merge.drop('population', axis=1, inplace=True)
merge.drop('area_sq_km', axis=1, inplace=True)
merge.drop('population_density', axis=1, inplace=True)
merge.drop('gdp_per_capita_usd', axis=1, inplace=True)

merge

,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases_x,Total_confirmed_cases_y,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested
0,Colombia,COL,40827302,2021-03-14,2728,2728,54597,6306,30968
1,Colombia,COL,40827302,2021-03-15,4440,4440,65338,5419,41720
2,Colombia,COL,40827302,2021-03-16,5093,5093,59933,2243,34063
3,Colombia,COL,40827302,2021-03-17,5802,5802,42313,5908,46016
4,Colombia,COL,40827302,2021-03-18,6591,6591,51364,774,43986
...,...,...,...,...,...,...,...,...,...
31513,South Africa,ZAF,39149717,2022-06-09,1591,1591,16404,16781,20865
31514,South Africa,ZAF,39149717,2022-06-13,710,710,838,854,10493
31515,South Africa,ZAF,39149717,2022-06-14,1903,1903,14113,13788,22417
31516,South Africa,ZAF,39149717,2022-06-20,1087,1087,566,654,14018


In [85]:
# 6.1 world_countries.csv 

# Tutaj możemy usunąć dane z 4.1, ponieważ są one już w pliku .csv z 4.1 

world_countries = pd.read_csv("world_countries.csv")

merge = pd.merge(merge, world_countries, left_on=['3_letter_code'], right_on=['CCA3'], how="left")
merge.drop('Country/Territory', axis=1, inplace=True)
merge.drop('CCA3', axis=1, inplace=True)

merge

,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases_x,Total_confirmed_cases_y,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,Rank,...,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage
0,Colombia,COL,40827302,2021-03-14,2728,2728,54597,6306,30968,28,...,47119728,44816108,39215135,32601393,26176195,20905254,1141748,45.4339,1.0069,0.65
1,Colombia,COL,40827302,2021-03-15,4440,4440,65338,5419,41720,28,...,47119728,44816108,39215135,32601393,26176195,20905254,1141748,45.4339,1.0069,0.65
2,Colombia,COL,40827302,2021-03-16,5093,5093,59933,2243,34063,28,...,47119728,44816108,39215135,32601393,26176195,20905254,1141748,45.4339,1.0069,0.65
3,Colombia,COL,40827302,2021-03-17,5802,5802,42313,5908,46016,28,...,47119728,44816108,39215135,32601393,26176195,20905254,1141748,45.4339,1.0069,0.65
4,Colombia,COL,40827302,2021-03-18,6591,6591,51364,774,43986,28,...,47119728,44816108,39215135,32601393,26176195,20905254,1141748,45.4339,1.0069,0.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31513,South Africa,ZAF,39149717,2022-06-09,1591,1591,16404,16781,20865,24,...,55876504,51784921,46813266,39877570,29463549,22368306,1221037,49.0517,1.0084,0.75
31514,South Africa,ZAF,39149717,2022-06-13,710,710,838,854,10493,24,...,55876504,51784921,46813266,39877570,29463549,22368306,1221037,49.0517,1.0084,0.75
31515,South Africa,ZAF,39149717,2022-06-14,1903,1903,14113,13788,22417,24,...,55876504,51784921,46813266,39877570,29463549,22368306,1221037,49.0517,1.0084,0.75
31516,South Africa,ZAF,39149717,2022-06-20,1087,1087,566,654,14018,24,...,55876504,51784921,46813266,39877570,29463549,22368306,1221037,49.0517,1.0084,0.75


In [86]:
gdp = pd.read_csv("gdp.csv")

merge = pd.merge(merge, gdp, left_on=['3_letter_code'], right_on=['Country Code'], how="left")
merge.drop('Country Name', axis=1, inplace=True)
merge.drop('Country Code', axis=1, inplace=True)

merge.to_csv("wyniki\\final.csv", index=False)

merge

,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases_x,Total_confirmed_cases_y,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,Rank,...,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,Year,Value
0,Colombia,COL,40827302,2021-03-14,2728,2728,54597,6306,30968,28,...,39215135,32601393,26176195,20905254,1141748,45.4339,1.0069,0.65,1960.0,4.040948e+09
1,Colombia,COL,40827302,2021-03-14,2728,2728,54597,6306,30968,28,...,39215135,32601393,26176195,20905254,1141748,45.4339,1.0069,0.65,1961.0,4.552914e+09
2,Colombia,COL,40827302,2021-03-14,2728,2728,54597,6306,30968,28,...,39215135,32601393,26176195,20905254,1141748,45.4339,1.0069,0.65,1962.0,4.968604e+09
3,Colombia,COL,40827302,2021-03-14,2728,2728,54597,6306,30968,28,...,39215135,32601393,26176195,20905254,1141748,45.4339,1.0069,0.65,1963.0,4.838841e+09
4,Colombia,COL,40827302,2021-03-14,2728,2728,54597,6306,30968,28,...,39215135,32601393,26176195,20905254,1141748,45.4339,1.0069,0.65,1964.0,5.992169e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500224,South Africa,ZAF,39149717,2022-06-21,1028,1028,12609,13165,19244,24,...,46813266,39877570,29463549,22368306,1221037,49.0517,1.0084,0.75,2012.0,3.963326e+11
1500225,South Africa,ZAF,39149717,2022-06-21,1028,1028,12609,13165,19244,24,...,46813266,39877570,29463549,22368306,1221037,49.0517,1.0084,0.75,2013.0,3.668100e+11
1500226,South Africa,ZAF,39149717,2022-06-21,1028,1028,12609,13165,19244,24,...,46813266,39877570,29463549,22368306,1221037,49.0517,1.0084,0.75,2014.0,3.511191e+11
1500227,South Africa,ZAF,39149717,2022-06-21,1028,1028,12609,13165,19244,24,...,46813266,39877570,29463549,22368306,1221037,49.0517,1.0084,0.75,2015.0,3.176107e+11


# Część 7

In [87]:
# https://data.worldbank.org/indicator/FP.CPI.TOTL.ZG?end=2023&start=1998

inflation_csv = pd.read_csv("Inflation\\inflation.csv")

inflation_csv = inflation_csv[['Country Name', 'Country Code', '2020 [YR2020]', '2021 [YR2021]', '2022 [YR2022]']]

inflation_csv.dropna(inplace=True)
inflation_csv.tail()

merge6 = pd.merge(merge, inflation_csv, left_on=['3_letter_code'], right_on=['Country Code'] , how='left',)
merge6.drop('Country Code', axis=1, inplace=True)
merge6.drop('Country Name', axis=1, inplace=True)
merge6


,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases_x,Total_confirmed_cases_y,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,Rank,...,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,Year,Value,2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,Colombia,COL,40827302,2021-03-14,2728,2728,54597,6306,30968,28,...,20905254,1141748,45.4339,1.0069,0.65,1960.0,4.040948e+09,2.52663500084706,3.49505757399448,10.1772313546166
1,Colombia,COL,40827302,2021-03-14,2728,2728,54597,6306,30968,28,...,20905254,1141748,45.4339,1.0069,0.65,1961.0,4.552914e+09,2.52663500084706,3.49505757399448,10.1772313546166
2,Colombia,COL,40827302,2021-03-14,2728,2728,54597,6306,30968,28,...,20905254,1141748,45.4339,1.0069,0.65,1962.0,4.968604e+09,2.52663500084706,3.49505757399448,10.1772313546166
3,Colombia,COL,40827302,2021-03-14,2728,2728,54597,6306,30968,28,...,20905254,1141748,45.4339,1.0069,0.65,1963.0,4.838841e+09,2.52663500084706,3.49505757399448,10.1772313546166
4,Colombia,COL,40827302,2021-03-14,2728,2728,54597,6306,30968,28,...,20905254,1141748,45.4339,1.0069,0.65,1964.0,5.992169e+09,2.52663500084706,3.49505757399448,10.1772313546166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500224,South Africa,ZAF,39149717,2022-06-21,1028,1028,12609,13165,19244,24,...,22368306,1221037,49.0517,1.0084,0.75,2012.0,3.963326e+11,3.21003597454107,4.61167217803206,7.03972661255866
1500225,South Africa,ZAF,39149717,2022-06-21,1028,1028,12609,13165,19244,24,...,22368306,1221037,49.0517,1.0084,0.75,2013.0,3.668100e+11,3.21003597454107,4.61167217803206,7.03972661255866
1500226,South Africa,ZAF,39149717,2022-06-21,1028,1028,12609,13165,19244,24,...,22368306,1221037,49.0517,1.0084,0.75,2014.0,3.511191e+11,3.21003597454107,4.61167217803206,7.03972661255866
1500227,South Africa,ZAF,39149717,2022-06-21,1028,1028,12609,13165,19244,24,...,22368306,1221037,49.0517,1.0084,0.75,2015.0,3.176107e+11,3.21003597454107,4.61167217803206,7.03972661255866
